In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy
from scipy import stats
import mygene
import math

import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')
% matplotlib inline

## Creating Background Network ###

#### scratch code

In [2]:
# load TR list from AnimalTFDB 
TR_db_m = pd.read_csv("Mus_musculus_transcription_factors_gene_list.txt", sep = "\t")
TR_db_h = pd.read_csv("Homo_sapiens_transcription_factors_gene_list.txt", sep = "\t")
TR_db = TR_db_m.append(TR_db_h)
TR_list_entrez = TR_db.Entrez_ID

In [3]:
len(TR_db_h)

1691

In [4]:
#translate TR list entrez to symbol
mg = mygene.MyGeneInfo()
translated_DF = mg.getgenes(set(TR_list_entrez), as_dataframe=True)
animal_TF = translated_DF["symbol"].str.upper()
len(animal_TF)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-2956...done.


2956

In [ ]:
#DEG_list = [7,9,10]
#edge_list = [(2,5), (2,4), (1,5), (3,5), (3,4), (6,4), (6,7), (6,9), (6,10), (8,9), (8,10)]
#DG = nx.DiGraph()
#DG.add_edges_from(edge_list)
#sym1_list = [2, 2, 1, 3, 3, 6, 6, 6, 6, 8, 8]
#sym2_list = [5, 4, 5, 5, 4, 4, 7, 9, 10, 9, 10]
#source_nodes = list(set(zip(*DG.edges())[0]))
#print 'source_nodes: ' + str(source_nodes)

### Real code starts here ###

In [76]:
def load_slowkow(filename_list = ['./slowkow_databases/TRED_TF.txt',
                 './slowkow_databases/ITFP_TF.txt',
                 './slowkow_databases/ENCODE_TF.txt',
                 './slowkow_databases/Neph2012_TF.txt',
                 './slowkow_databases/TRRUST_TF.txt',
                  './slowkow_databases/Marbach2016_TF.txt']):
    
    # read files formatted as \n separated items
    return_list = []
    for file_name in filename_list:
        with open(file_name) as f:
            lines = f.read().splitlines()
            return_list.extend(lines)
    
    # convert everything to ALL CAPS
    [x.upper() for x in return_list]
    
    # remove duplicates
    return set(return_list)

filename_list = ['./slowkow_databases/TRED_TF.txt',
                 './slowkow_databases/ITFP_TF.txt',
                 './slowkow_databases/ENCODE_TF.txt',
                 './slowkow_databases/Neph2012_TF.txt',
                 './slowkow_databases/TRRUST_TF.txt',
                  './slowkow_databases/Marbach2016_TF.txt']
len(load_slowkow(filename_list))

2705

In [24]:
def load_jaspar(filename):
    
    # parse jaspar file
    jasp_df = pd.read_csv(filename, sep = "\t", header= None, names = ['col1', 'col2', 'col3', 'col4', 'tf_genes'])
    
    # return transcription factors with ALL CAPS names
    return list(jasp_df['tf_genes'].str.upper())
    
    
len(load_jaspar("jaspar_genereg_matrix.txt"))

2049

In [25]:
def create_TF_list(slowkow_bool = True,
                   slowkow_files = ['./slowkow_databases/TRED_TF.txt',
                                    './slowkow_databases/ITFP_TF.txt',
                                    './slowkow_databases/ENCODE_TF.txt',
                                    './slowkow_databases/Neph2012_TF.txt',
                                    './slowkow_databases/TRRUST_TF.txt',
                                    './slowkow_databases/Marbach2016_TF.txt'],
                   jaspar_bool = True, 
                   jaspar_file = "jaspar_genereg_matrix.txt"):
    
    TF_list = []
    
    if slowkow_bool == True:
        slowkow_TFs = load_slowkow(slowkow_files)
        TF_list.extend(slowkow_TFs)
        
    if jaspar_bool == True:
        jaspar_TFs = load_jaspar(jaspar_file)
        TF_list.extend(jaspar_TFs)
        
    return list(set(TF_list))
        
        
TF_list = create_TF_list()
len(TF_list)

3983

#### Cross reference Brin's TR list with background STRING db


In [79]:
def load_and_process_STRING(filename = "STRING_network.xlsx"):
    
    # Load STRING database as background network
    STRING_DF = pd.read_excel(filename)

    # convert sources (sym1) and targets (sym2) to all caps
    sym1_list = STRING_DF.Symbol1.str.upper()
    sym2_list = STRING_DF.Symbol2.str.upper()

    # make an edge list with associated edge weight (db_edges)
    weight_list = STRING_DF.Weight
    db_edges = zip(sym1_list, sym2_list, weight_list)

    # make an edge list with associated activating (+)/inhibiting (-) sign (db_sign_att)
    sign_list = STRING_DF.Edge_Sign
    sign_num_list = []
    for sign in sign_list:
        if str(sign) == '+':
            sign_num_list.append(1)
        elif str(sign) == '-':
            sign_num_list.append(-1)
        else:
            sign_num_list.append(0)
    db_sign_att = zip(sym1_list, sym2_list, sign_num_list)
    
    return STRING_DF, db_edges, db_sign_att


STRING_DF, db_edges, db_sign_att = load_and_process_STRING();

In [29]:
def filter_background(db_edges, db_sign_att, TF_list):
    
    # extracting TR edge information from background database
    edge_list_filtered = []
    sign_att_list_filtered = []
    for i in range(len(db_edges)):
        if db_edges[i][0] in list(TF_list):
            edge_list_filtered.append(db_edges[i])
            sign_att_list_filtered.append(db_sign_att[i])
            
    return edge_list_filtered, sign_att_list_filtered

edge_list_filtered, sign_att_list_filtered = filter_background(db_edges, db_sign_att, TF_list)

In [32]:
def make_digraph(db_edges, db_sign_att, TF_list):
    
    # use only edges from background network associated with our TF list
    edge_list_filtered, sign_att_list_filtered = filter_background(db_edges, db_sign_att, TF_list)
    
    # create networkx digraph from weighted edge list, add sign edge attributes 
    DG = nx.DiGraph()
    DG.add_weighted_edges_from(edge_list_filtered)
    for i in range(len(sign_att_list_filtered)):
        DG[sign_att_list_filtered[i][0]][sign_att_list_filtered[i][1]]['sign'] = sign_att_list_filtered[i][2]
    
    return DG

DG = make_digraph(db_edges, db_sign_att, TF_list)

In [33]:
len(DG.nodes())

687

### p-value with differencially expressed genes

In [34]:
def load_DEG_with_up_downs(filename = "differencially_expressed_genes.txt", filter_value = 0.3):

    # load differencially expressed genes (experimental results)
    DEG_db = pd.read_csv(filename, sep = "\t")

    # filtering for lfdr < 0.3
    DEG_list = []
    DEG_to_updown = {}
    for i in range(len(DEG_db)):

        # removing Nan values
        if str(DEG_db.symbol[i]).upper() != 'NAN':

            # filtering DEG list by lfdr < filter_value
            if (DEG_db['lfdr.89.12'][i] < filter_value):
                DEG_list.append(str(DEG_db.symbol[i]).upper())

                # creating dictionary between DEG symbols and their up/down value
                if DEG_db['log2.89.12'][i] != 0: 
                    DEG_to_updown[str(DEG_db.symbol[i]).upper()] = DEG_db['log2.89.12'][i]
                else:
                    DEG_to_updown[str(DEG_db.symbol[i]).upper()] = 0
    
    return DEG_list, DEG_to_updown

DEG_list, DEG_to_updown = load_DEG_with_up_downs()
print len(DEG_list)
print len(DEG_to_updown)

2782
2782


In [42]:
def add_updown_from_DEG(DG, DEG_filename = "differencially_expressed_genes.txt", DEG_filter_value = 0.3):
    
    DEG_list, DEG_to_updown = load_DEG_with_up_downs(DEG_filename, DEG_filter_value)
    
    # get all the differencially expressed genes in DG
    DEG_in_DG = set(DG.nodes()) & set(DEG_list)
    
    # add node attribute to each node in DG if it exists, otherwise set to zero
    zero_dict = dict(zip(DG.nodes(), [0]*len(DG.nodes())))
    for gene in DEG_in_DG:
        zero_dict[gene] = DEG_to_updown[gene]
    nx.set_node_attributes(DG, 'updown', zero_dict)
    
    return DEG_list

In [43]:
DEG_list = add_updown_from_DEG(DG)
DG.nodes(data = True)[0:20]

[(u'PDM2', {'updown': 0}),
 (u'ICLN', {'updown': 0}),
 (u'MCM10', {'updown': 0}),
 (u'CG30085', {'updown': 0}),
 (u'VPS2', {'updown': 0}),
 (u'SCB', {'updown': 0}),
 (u'SIN', {'updown': 0}),
 (u'BAP60', {'updown': 0}),
 (u'VPS4', {'updown': 0}),
 (u'SMG5', {'updown': 0.18674921190000002}),
 (u'SPZ', {'updown': 0}),
 (u'SMG6', {'updown': 0.1827090375}),
 (u'TAP', {'updown': 0}),
 (u'RPS19B', {'updown': 0}),
 (u'RPS19A', {'updown': 0}),
 (u'TAZ', {'updown': 0}),
 (u'MRPL51', {'updown': 0}),
 (u'PROSBETA5R', {'updown': 0}),
 (u'RPT3', {'updown': 0}),
 (u'RPT1', {'updown': 0})]

In [74]:
# calculating all the p-scores

def tr_pvalues(DG, db_edges, DEG_list):
    
    source_nodes = list(set(zip(*DG.edges())[0]))  #identifying unique source nodes in graph
    background_list = list(set(zip(*db_edges)[0]) | set(zip(*db_edges)[1]))
    
    TR_to_pvalue = {}
    for TR in source_nodes:
        x = len(list(set(DG.neighbors(TR)) & set(DEG_list))) # per TR, observed overlap between TR neighbors and DEG_list
        M = len(background_list)  # num unique nodes in universe, aka background network (STRING)
        n = len(DG.neighbors(TR)) # per TR, number of targets for that TR
        N = len(list(set(background_list) & set(DEG_list))) # number of DEG, picked from universe "at random"
    
        TR_to_pvalue[TR] = -(scipy.stats.hypergeom.logsf(x, M, n, N, loc=0)) # remove unnecessary negative sign
        
    return TR_to_pvalue 
    
TR_to_pvalue = tr_pvalues(DG, db_edges, DEG_list)
len(TR_to_pvalue)

102

### z-score with DEG

In [75]:
def tr_zscore(DG, DEG_list):
    
    source_nodes = list(set(zip(*DG.edges())[0])) #identifying unique source nodes in graph
    
    TR_to_zscore = {}
    for TR in source_nodes:
        N_minus = 0 # number of inhibiting predicting DEG edges
        N_plus = 0 # number of activating predicting DEG edges
        N_zero = 0 # number of edges with errorous calculations
    
        TRs_DEG_neighbors = set(DG.neighbors(TR)) & set(DEG_list)
        for n in TRs_DEG_neighbors:
                sign_of_edge = DG[TR][n]['sign']
                up_down_of_n = (DG.node[n]['updown']/abs(DG.node[n]['updown']))
                
                # predict whether this neighbor thinks the TR is Act. or Inhib.
                if ((sign_of_edge * up_down_of_n) == 1):
                    N_plus += 1
                elif ((sign_of_edge * up_down_of_n) == -1):
                    N_minus += 1
                else:
                    N_zero += 1 # mark an error if could not predict
                    print "Issue with edge (" + str(TR) + ',' + str(n) + ')'
                
        if N_zero != 0:
            print "Could not attribute activated or inhibiting trait to " + str(N_zero) + 'nodes'
      
        # prevent a divide-by-zero calculation
        N = N_plus + N_minus
        if N == 0:
            z_score = 0
        else:
            z_score = (N_plus - N_minus)/float(math.sqrt(N))
                
        TR_to_zscore[TR] = z_score #create zscore dict where 1 means activating
                                                            # -1 means inhibiting
                                                            # 0 means could not be calculated
    
    return TR_to_zscore

tr_zscore(DG, DEG_list)

{u'ABD-B': 0,
 u'ACHI': 0,
 u'AKT1': 0,
 u'ANTP': 0,
 u'ARR1': 0,
 u'ATF6': 0,
 u'BAP': 0,
 u'BUB3': 0,
 u'CDC16': -1.0,
 u'CDC27': -1.0,
 u'CDC6': -1.0,
 u'CG11294': 0,
 u'CNOT4': 0,
 u'DFD': 0,
 u'DL': 0.0,
 u'ECD': 0,
 u'EVE': 0,
 u'FKH': 0,
 u'GATA': 0,
 u'GCM': 0,
 u'GSTO1': 0,
 u'H': 0,
 u'HBN': 0,
 u'HDAC3': 0,
 u'HDAC6': 0,
 u'HKB': 0,
 u'ILK': 0,
 u'ING3': 0,
 u'INR': 0,
 u'INTS4': -1.0,
 u'INTS6': -1.0,
 u'INTS8': -1.0,
 u'KLHL18': 0,
 u'KR': 0,
 u'LIG3': 0,
 u'MAD': 0,
 u'MARS': 0,
 u'MAX': 0,
 u'MCM2': -1.414213562373095,
 u'MCM3': -1.414213562373095,
 u'MCM5': -1.0,
 u'MCM6': -0.5773502691896258,
 u'MCM7': -1.0,
 u'MED1': -1.0,
 u'MED15': -1.414213562373095,
 u'MRPL23': 0,
 u'MRPL24': -1.7320508075688774,
 u'MRPL44': 0,
 u'MSH6': 0,
 u'MYB': 0,
 u'NF1': 0,
 u'NUB': 0,
 u'NUP133': 0,
 u'NUP50': 0,
 u'ONECUT': 0,
 u'OPTIX': 0,
 u'PAN': 0,
 u'PAX': 0.0,
 u'PNR': 0,
 u'PTEN': -1.0,
 u'PXN': 0,
 u'RAE1': 0,
 u'RBBP5': 0,
 u'REL': -1.0,
 u'REPO': 0,
 u'RFC4': -1.0,
 u'RNPS1': -1

## Testing

In [80]:
TF_list = create_TF_list()
STRING_DF, db_edges, db_sign_att = load_and_process_STRING()
edge_list_filtered, sign_att_list_filtered = filter_background(db_edges, db_sign_att, TF_list)
DG = make_digraph(db_edges, db_sign_att, TF_list)
DEG_list = add_updown_from_DEG(DG)
tr_pvalues(DG, db_edges, DEG_list)

{u'ABD-B': 2.5619299458469871,
 u'ACHI': 4.1400668115618942,
 u'AKT1': 2.0051353444087323,
 u'ANTP': 4.1400668115618942,
 u'ARR1': 3.4547738088253737,
 u'ATF6': 1.6680928067230729,
 u'BAP': 2.3873854072043095,
 u'BUB3': 1.2902562879815764,
 u'CDC16': 3.1441987530558704,
 u'CDC27': 2.0214533281293834,
 u'CDC6': 3.909551880959754,
 u'CG11294': 3.4547738088253737,
 u'CNOT4': 2.0051353444087323,
 u'DFD': 4.1400668115618942,
 u'DL': 5.3485051248405746,
 u'ECD': 2.5619299458469871,
 u'EVE': 4.1400668115618942,
 u'FKH': 4.1400668115618942,
 u'GATA': 4.1400668115618942,
 u'GCM': 4.1400668115618942,
 u'GSTO1': 4.1400668115618942,
 u'H': 2.3873854072043095,
 u'HBN': 4.1400668115618942,
 u'HDAC3': 2.3873854072043095,
 u'HDAC6': 2.7772771668834308,
 u'HKB': 3.4547738088253737,
 u'ILK': 3.4547738088253737,
 u'ING3': 2.3873854072043095,
 u'INR': 1.9074744647345609,
 u'INTS4': 5.6306387511600979,
 u'INTS6': 6.0258370197540474,
 u'INTS8': 6.5263844837145308,
 u'KLHL18': 1.8198445447027656,
 u'KR': 4.1

In [81]:
tr_zscore(DG, DEG_list)

{u'ABD-B': 0,
 u'ACHI': 0,
 u'AKT1': 0,
 u'ANTP': 0,
 u'ARR1': 0,
 u'ATF6': 0,
 u'BAP': 0,
 u'BUB3': 0,
 u'CDC16': -1.0,
 u'CDC27': -1.0,
 u'CDC6': -1.0,
 u'CG11294': 0,
 u'CNOT4': 0,
 u'DFD': 0,
 u'DL': 0.0,
 u'ECD': 0,
 u'EVE': 0,
 u'FKH': 0,
 u'GATA': 0,
 u'GCM': 0,
 u'GSTO1': 0,
 u'H': 0,
 u'HBN': 0,
 u'HDAC3': 0,
 u'HDAC6': 0,
 u'HKB': 0,
 u'ILK': 0,
 u'ING3': 0,
 u'INR': 0,
 u'INTS4': -1.0,
 u'INTS6': -1.0,
 u'INTS8': -1.0,
 u'KLHL18': 0,
 u'KR': 0,
 u'LIG3': 0,
 u'MAD': 0,
 u'MARS': 0,
 u'MAX': 0,
 u'MCM2': -1.414213562373095,
 u'MCM3': -1.414213562373095,
 u'MCM5': -1.0,
 u'MCM6': -0.5773502691896258,
 u'MCM7': -1.0,
 u'MED1': -1.0,
 u'MED15': -1.414213562373095,
 u'MRPL23': 0,
 u'MRPL24': -1.7320508075688774,
 u'MRPL44': 0,
 u'MSH6': 0,
 u'MYB': 0,
 u'NF1': 0,
 u'NUB': 0,
 u'NUP133': 0,
 u'NUP50': 0,
 u'ONECUT': 0,
 u'OPTIX': 0,
 u'PAN': 0,
 u'PAX': 0.0,
 u'PNR': 0,
 u'PTEN': -1.0,
 u'PXN': 0,
 u'RAE1': 0,
 u'RBBP5': 0,
 u'REL': -1.0,
 u'REPO': 0,
 u'RFC4': -1.0,
 u'RNPS1': -1